In [ ]:
import pandas as pd
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import random

In [ ]:
df=pd.read_csv('/content/brca_cancer (1).csv')

In [ ]:
df.head()

,Unnamed: 0,gene_id,gene_name,gene_type,unstranded,stranded_first,stranded_second,tpm_unstranded,fpkm_unstranded,fpkm_uq_unstranded,class
0,4,ENSG00000000003.15,TSPAN6,protein_coding,5175.0,2582.0,2594.0,58.4397,16.4625,16.0650,0.0
1,5,ENSG00000000005.6,TNMD,protein_coding,85.0,52.0,33.0,2.9499,0.8310,0.8109,0.0
2,6,ENSG00000000419.13,DPM1,protein_coding,2406.0,1194.0,1212.0,102.1078,28.7639,28.0692,0.0
3,7,ENSG00000000457.14,SCYL3,protein_coding,2222.0,1629.0,1631.0,16.5362,4.6583,4.5458,0.0
4,8,ENSG00000000460.17,C1orf112,protein_coding,476.0,808.0,782.0,4.0842,1.1505,1.1227,0.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121320 entries, 0 to 121319
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Unnamed: 0          121320 non-null  int64  
 1   gene_id             121320 non-null  object 
 2   gene_name           121320 non-null  object 
 3   gene_type           121320 non-null  object 
 4   unstranded          121320 non-null  int64  
 5   stranded_first      121320 non-null  int64  
 6   stranded_second     121320 non-null  int64  
 7   tpm_unstranded      121320 non-null  float64
 8   fpkm_unstranded     121320 non-null  float64
 9   fpkm_uq_unstranded  121320 non-null  float64
 10  class               121320 non-null  int64  
dtypes: float64(3), int64(5), object(3)
memory usage: 10.2+ MB


In [ ]:
dfe=df.loc[:,['gene_name','tpm_unstranded','class']]

In [ ]:
dfe.dropna(inplace=True)

In [ ]:
dfe.head()

,gene_name,tpm_unstranded,class
0,56562,58.4397,0.0
1,55779,2.9499,0.0
2,26531,102.1078,0.0
3,51532,16.5362,0.0
4,22695,4.0842,0.0


In [ ]:
len(dfe['gene_name'].unique())

59427

In [ ]:
dfe['gene_name'].value_counts()

gene_name
58185    756
40316    170
56915     50
56923     33
53051     27
        ... 
54043      1
31324      1
15807      1
17028      1
20927      1
Name: count, Length: 59427, dtype: int64

In [ ]:
dfe[dfe['gene_name']=='PIK3CA']

,gene_name,tpm_unstranded,class


In [ ]:
dfe[dfe['gene_name']=='TP53']

,gene_name,tpm_unstranded,class


In [ ]:
dfe[dfe['gene_name']=='BRCA1']

,gene_name,tpm_unstranded,class


In [ ]:
dfe[dfe['gene_name']=='NF1']

,gene_name,tpm_unstranded,class


In [ ]:

from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

In [ ]:

dfe['gene_name']=encoder.fit_transform(dfe['gene_name'])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
y=dfe['class']
X=dfe.drop(['class'],axis=1)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=101)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

In [ ]:
dtree=DecisionTreeClassifier()

In [ ]:
dtree.fit(X_train,y_train)

DecisionTreeClassifier()

In [ ]:
y_predictions=dtree.predict(X_test)

In [ ]:
print(classification_report(y_test,y_predictions))

              precision    recall  f1-score   support

         0.0       0.89      0.88      0.89     18259
         1.0       0.27      0.29      0.28      2770

    accuracy                           0.80     21029
   macro avg       0.58      0.59      0.58     21029
weighted avg       0.81      0.80      0.81     21029



In [ ]:
def notSoFit(selected_genes):
    independentSelected = df[selected_genes]
    y = df['class']
    X_train, X_test, y_train, y_test = train_test_split(independentSelected, y, test_size=0.3, random_state=101)
    dtree = DecisionTreeClassifier()
    try:
        dtree.fit(X_train, y_train)
        y_pred = dtree.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
    except ValueError:
        accuracy = 0
    return accuracy

In [ ]:
def geneticAlgo(populationSize, generations, mutationRate, genePool):
    population = []
    for _ in range(populationSize):
#_______________________________________________________________________________________
#Random Chromosomes
        chromosome = random.choices(genePool, k=30)
        population.append(chromosome)
#_______________________________________________________________________________________
#Chromosome Fitness Score
    for _ in range(generations):
        notSoFitScore = [notSoFit(chromosome) for chromosome in population]

#Fitness Score
        if all(score == 0 for score in notSoFitScore):
#if zero then set to uniform
            probabilities = [1 / populationSize] * populationSize
        else:
#selection probalilites
            probabilities = np.square(notSoFitScore) / np.sum(np.square(notSoFitScore))

#_______________________________________________________________________________________
#Parents from generation
        parents = []
        for _ in range(populationSize // 2):
            idx1 = np.random.choice(range(populationSize), p=probabilities)
            idx2 = np.random.choice(range(populationSize), p=probabilities)
            parents.append((population[idx1], population[idx2]))
#_______________________________________________________________________________________
#Crossover
        new_population = []
        for parent1, parent2 in parents:
            crossOverPoint = np.random.randint(1, 29)
            child1 = parent1[:crossOverPoint] + parent2[crossOverPoint:]
            child2 = parent2[:crossOverPoint] + parent1[crossOverPoint:]
            new_population.extend([child1, child2])
#_______________________________________________________________________________________
#Mutation

        for i in range(populationSize):
            if np.random.rand() < mutationRate:
                reslutingMutatedGene = np.random.choice(genePool)
                mutationPoint = np.random.randint(30)
                population[i][mutationPoint] = reslutingMutatedGene

        population = new_population
#_______________________________________________________________________________________
#Best Chromosome
    mereJaisaChromosomes = max(population, key=lambda x: notSoFit(x))
    mereJaisaFitness = notSoFit(mereJaisaChromosomes)

    return mereJaisaChromosomes, mereJaisaFitness

In [ ]:
populationSize = 50

In [ ]:
generations = 100

In [ ]:
mutationRate = 0.1

In [ ]:
genePool = df.columns[:-1]

In [ ]:
mereJaisaChromosomes, mereJaisaFitness = geneticAlgo(populationSize, generations, mutationRate, genePool)

In [ ]:
print("Selected genes:")
print(mereJaisaChromosomes)
print("Best fitness (accuracy):", mereJaisaFitness)

Selected genes:
['unstranded', 'gene_name', 'gene_name', 'tpm_unstranded', 'tpm_unstranded', 'stranded_second', 'gene_name', 'stranded_second', 'tpm_unstranded', 'gene_name', 'Unnamed: 0', 'gene_id', 'fpkm_unstranded', 'stranded_second', 'fpkm_unstranded', 'fpkm_unstranded', 'gene_id', 'fpkm_unstranded', 'Unnamed: 0', 'gene_id', 'unstranded', 'stranded_first', 'gene_type', 'gene_id', 'gene_name', 'stranded_second', 'stranded_first', 'gene_type', 'stranded_second', 'gene_id']
Best fitness (accuracy): 0
